In [1]:
import numpy as np
import pandas as pd
from data_loader import load_names_from_web, holdout_split, year_split
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, accuracy_score
from scipy.stats import kendalltau

In [2]:
dfraw = load_names_from_web(category='national', hide_pre_1937=True, use_existing_files=True)
# traintestval, holdout = year_split(dfraw, holdout_size=30)
trainval, test = year_split(dfraw, holdout_size=30)

In [3]:
sorted(trainval['year'].unique())

[1937,
 1938,
 1939,
 1940,
 1941,
 1942,
 1943,
 1944,
 1945,
 1946,
 1947,
 1948,
 1949,
 1950,
 1951,
 1952,
 1953,
 1954,
 1955,
 1956,
 1957,
 1958,
 1959,
 1960,
 1961,
 1962,
 1963,
 1964,
 1965,
 1966,
 1967,
 1968,
 1969,
 1970,
 1971,
 1972,
 1973,
 1974,
 1975,
 1976,
 1977,
 1978,
 1979,
 1980,
 1981,
 1982,
 1983,
 1984,
 1985,
 1986,
 1987,
 1988,
 1989,
 1990,
 1991,
 1992]

In [4]:
def get_all_known_names(data):

    names = data.groupby(['state', 'name', 'M/F']).size().reset_index()
    names = names[['state', 'name', 'M/F']]
    return names

In [5]:
def select_top_names(data, first_year_to_predict, cutoff=10):
    '''
    Select names that have had at least one year at or above the cutoff count?
    Could also try to redo this to do the cutoff for the most recent known year's data
    '''

    data_noleak = data[data['year'] < first_year_to_predict]
    names = data_noleak[data_noleak['count'] >= cutoff][['state', 'name', 'M/F']].drop_duplicates()
    # display(names)
    data = names.merge(data, how='left', on=['state', 'name', 'M/F'])
    # display(data)

    return data

In [6]:
def classify(df):

    '''
    PEAKS
    '''

    peaks = df.loc[df.groupby(['state', 'name', 'M/F'])['count'].idxmax()][['state', 'name', 'M/F', 'year']]
    peaks = peaks.rename(columns={'year': 'peak_year'})
    df = df.merge(peaks, how='left', on=['state', 'name', 'M/F'])
    # display(df)
    df['has_peaked'] = df.apply(lambda row: 1 if row['year'] >= row['peak_year'] else 0, axis=1)
    return df

In [7]:
example = classify(trainval)
example[(example['peak_year'] == 1984) & (example['year'] == 1983)].sort_values(by='count', ascending=False).head(10)

,state,year,name,M/F,count,peak_year,has_peaked
655103,US,1983,Christopher,M,59340,1984,0
655119,US,1983,Adam,M,23489,1984,0
643065,US,1983,Lindsey,F,8681,1984,0
643071,US,1983,Alicia,F,7491,1984,0
655159,US,1983,Marcus,M,5634,1984,0
643120,US,1983,Latoya,F,3151,1984,0
643137,US,1983,Sheena,F,2634,1984,0
643158,US,1983,Lacey,F,2383,1984,0
643169,US,1983,Joanna,F,2120,1984,0
643229,US,1983,Kate,F,1334,1984,0


In [8]:
def evaluate(predictor, data_held_out, first_year_to_predict, metric='accuracy'):

    # display('data_held_out:')
    # display(data_held_out)

    # most_recent_year = data_held_out['year'].max()
    most_recent_year = first_year_to_predict - 1
    
    # years_to_predict = range(first_year_to_predict, most_recent_year+1)

    # only allow the model to see data from before the year to predict
    historical_data = data_held_out[data_held_out['year'] < first_year_to_predict]

    # display('historical_data:')
    # display(historical_data)

    # get our model's predictions
    predictions = predictor.predict(historical_data)

    all_known_names = get_all_known_names(historical_data)

    # now we need to know the "answers" -- what the real classes are, based on information in the "future"
    data_held_out = classify(data_held_out)
    data_held_out = data_held_out[data_held_out['year'] == most_recent_year]
    # print(f'Predictions for {year_to_predict}:')

    # names_to_predict = all_known_names.copy()
    # names_to_predict['year'] = year_to_predict
    # display(names_to_predict)

    observed = all_known_names.merge(data_held_out, how='left', on=['state', 'name', 'M/F'])

    observed['y'] = observed['has_peaked'].fillna(1) # if nothing for this name for this year, it has already peaked, so 1
    # display(observed)

    score_df = observed.merge(predictions, how='left', on=['state', 'name', 'M/F'], suffixes=('_true', '_pred'))
    # display(score_df)

    y_true = score_df['y_true']
    y_pred = score_df['y_pred']

    '''
    if metric == 'msle':
        loss = mean_squared_log_error(y_true, y_pred)
        print(f'Loss: {loss}')

    if metric == 'rank':
        y_true = y_true.rank()
        y_pred = y_pred.rank()
        score = np.sum(np.abs(y_true-y_pred))/(len(y_true)*(len(y_true)-1))
        print(f'Score: {score}')

    if metric == 'rank_mae':
        y_true = y_true.rank()
        y_pred = y_pred.rank()
        loss = mean_absolute_error(y_true, y_pred)
        print(f'Loss: {loss}')

    if metric == 'kendalltau':
        y_true = y_true.rank()
        y_pred = y_pred.rank()
        tau, _ = kendalltau(y_true, y_pred)
        print(f'Tau: {tau}')
    '''

    if metric == 'accuracy':
        score = accuracy_score(y_true, y_pred)
        print(f'Score: {score}')
    
    '''
    top_F_true = score_df[score_df['M/F'] == 'F'][['name', 'y_true']].sort_values(by='y_true', ascending=False).reset_index(drop=True)
    top_F_pred = score_df[score_df['M/F'] == 'F'][['name', 'y_pred']].sort_values(by='y_pred', ascending=False).reset_index(drop=True)
    top_M_true = score_df[score_df['M/F'] == 'M'][['name', 'y_true']].sort_values(by='y_true', ascending=False).reset_index(drop=True)
    top_M_pred = score_df[score_df['M/F'] == 'M'][['name', 'y_pred']].sort_values(by='y_pred', ascending=False).reset_index(drop=True)
    top = pd.concat([top_F_true, top_F_pred, top_M_true, top_M_pred], axis=1, ignore_index=True)
    '''
    display(score_df)

In [9]:
class DummyClassifier():

    def __init__(self, strategy='after_peak'):
        self.strategy = strategy
    
    def predict(self, historical_data):

        all_known_names = get_all_known_names(historical_data)

        most_recent_year = historical_data['year'].max()

        predictions = all_known_names.copy()

        if self.strategy == 'constant':
            predictions['y'] = 1
        elif self.strategy == 'after_peak':
            historical_data = classify(historical_data)
            most_recent_year_data = historical_data[historical_data['year'] == most_recent_year].drop(columns=['year'])
            predictions = predictions.merge(most_recent_year_data, how='left', on=['state', 'name', 'M/F'])
            predictions['peak_year'] = predictions['peak_year'].fillna(0)
            predictions['y'] = predictions['peak_year'].apply(lambda x: 1 if x < most_recent_year else 0)
            # display(predictions)

        return predictions

In [17]:
first_year_to_predict = 1993
cutoff = 100
# data_to_fit = select_top_names(trainval, first_year_to_predict=first_year_to_predict, cutoff=0)
data_to_eval = select_top_names(test, first_year_to_predict=first_year_to_predict, cutoff=cutoff)
evaluate(predictor=DummyClassifier(strategy='after_peak'), data_held_out=data_to_eval, first_year_to_predict=first_year_to_predict, metric='accuracy')

Score: 0.8126976954360596


,state,name,M/F,year,count_true,peak_year_true,has_peaked_true,y_true,count_pred,peak_year_pred,has_peaked_pred,y_pred
0,US,Aaron,F,1992.0,91.0,1980.0,1.0,1.0,91.0,1980.0,1.0,1
1,US,Aaron,M,1992.0,14509.0,1989.0,1.0,1.0,14509.0,1989.0,1.0,1
2,US,Abbey,F,1992.0,431.0,1999.0,0.0,0.0,431.0,1990.0,1.0,1
3,US,Abbie,F,1992.0,260.0,2003.0,0.0,0.0,260.0,1990.0,1.0,1
4,US,Abby,F,1992.0,1081.0,2003.0,0.0,0.0,1081.0,1983.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4421,US,Zelma,F,1992.0,8.0,1938.0,1.0,1.0,8.0,1938.0,1.0,1
4422,US,Zena,F,1992.0,52.0,1964.0,1.0,1.0,52.0,1964.0,1.0,1
4423,US,Zina,F,1992.0,21.0,1964.0,1.0,1.0,21.0,1964.0,1.0,1
4424,US,Zoe,F,1992.0,982.0,2012.0,0.0,0.0,982.0,1992.0,1.0,0


In [11]:
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import HistGradientBoostingRegressor, HistGradientBoostingClassifier

class MyPredictor():

    def __init__(self):
        
        # params
        # cols_to_keep = ['this_year', 'M/F', 'sum', 'median_age', 'thisyear_count', 'diff',] # best overall?
        # cols_to_keep = ['this_year', 'M/F', 'sum', 'median_age', 'thisyear_count', 'diff', 'first_letter_1_pct', 'first_letter_2_pct', 'first_letter_3_pct'] # first letter pct change helps in some years
        # cols_to_keep = ['this_year', 'M/F', 'sum', 'median_age', 'thisyear_count', 'diff', 'thisyear_count_opp']
        cols_to_keep = ['M/F', 'sum', 'median_age', 'thisyear_count', 'diff', 'thisyear_count_opp', 'diff2', 'shift', 'pct_change', 'accel', 'after_peak', 'years_since_peak', 'first_letter_1_pct', 'first_letter_2_pct', 'first_letter_3_pct']
        categorical_features = ['M/F', 'after_peak']
        max_leaf_nodes = 31 # 16
        max_iter = 100 # 100
        loss = 'log_loss' # abs better than default

        categorical_features = [True if f in categorical_features else False for f in cols_to_keep]
        # print(categorical_features)
        
        self.pipe = make_pipeline(
            ColumnTransformer(
                transformers=[
                    # ('category_encoder', LabelEncoder(), categorical_features),
                    ('cols_to_keep', 'passthrough', cols_to_keep),
                ], remainder='drop'),
            HistGradientBoostingClassifier(
                random_state=0,
                categorical_features=categorical_features,
                max_leaf_nodes=max_leaf_nodes,
                max_iter=max_iter,
                loss=loss
            )
        )

        self.gender_encoding = {'M': 0, 'F': 1}

    def preprocess(self, df, latest_known_year):

        # print(f'Latest known year: {latest_known_year}')

        # find median age of people with name, 
        # total born with that name,
        # and latest year's count

        df = df.copy()

        df = classify(df)
        df['after_peak'] = df['peak_year'].apply(lambda x: 1 if x < latest_known_year else 0)

        df = df.sort_values(by='year')
        df['cumsum'] = df.groupby(['state', 'name', 'M/F'])['count'].cumsum()
        df['sum'] = df.groupby(['state', 'name', 'M/F'])['count'].transform('sum')

        df['diff'] = df.groupby(['state', 'name', 'M/F'])['count'].diff()
        df['diff2'] = df.groupby(['state', 'name', 'M/F'])['count'].diff(2)
        df['shift'] = df.groupby(['state', 'name', 'M/F'])['count'].shift()
        df['pct_change'] = df.groupby(['state', 'name', 'M/F'])['count'].pct_change()
        df['accel'] = df.groupby(['state', 'name', 'M/F'])['diff'].diff()

        percentage_of_total_per_year = {}
        percentage_change_per_year = {}

        def first_letters(df, n, percentage_of_total_per_year, percentage_change_per_year):
            df['first_letter_'+str(n)] = df['name'].str[0:n].str.lower()
            total_names_per_year = df.groupby(['year', 'state', 'M/F'])['count'].sum()
            letter_names_per_year = df.groupby(['year', 'state', 'M/F', 'first_letter_'+str(n)])['count'].sum()
            percentage_of_total_per_year[n] = (letter_names_per_year / total_names_per_year).rename('first_letter_'+str(n)+'_pct')
            # display(percentage_of_total_per_year)
            percentage_change_per_year[n] = percentage_of_total_per_year[n].groupby(['state', 'M/F', 'first_letter_'+str(n)]).pct_change().rename('first_letter_'+str(n)+'_pct_change')
            # display(percentage_change_per_year)
            return df
        
        for n in range(1, 4):
            df = first_letters(df, n, percentage_of_total_per_year, percentage_change_per_year)
        
        # display(df[(df['name'] == 'Maximus')])

        medians = df[df['cumsum'] >= df['sum']/2]
        medians = medians.drop_duplicates(subset=['state', 'name', 'M/F'], keep='first')
        medians['median_age'] = latest_known_year - medians['year']
        # display(medians[medians['name'] == 'Madison'])

        medians = medians.drop(['count', 'cumsum', 'diff', 'shift', 'pct_change', 'accel', 'diff2', 'after_peak'], axis=1)

        thisyear = df[df['year'] == latest_known_year][['state', 'name', 'M/F', 'count', 'diff', 'shift', 'pct_change', 'accel', 'diff2', 'after_peak']].rename(columns={'count': 'thisyear_count'})
        # thisyear = thisyear.merge(percentage_of_total_per_year, how='left', on=['year', 'state', 'M/F', 'first_letter_1'])
        # thisyear = thisyear.merge(percentage_change_per_year, how='left', on=['year', 'state', 'M/F', 'first_letter_1'])
        # thisyear = thisyear.rename(columns={'year': 'this_year'})

        sex_counts = thisyear.groupby(['state', 'name', 'M/F'])['thisyear_count'].sum()
        thisyear_swapped = thisyear.copy()
        thisyear_swapped['M/F'] = thisyear_swapped['M/F'].replace({'M': 'F', 'F': 'M'})
        thisyear_swapped = thisyear_swapped.merge(sex_counts, how='left', on=['state', 'name', 'M/F'], suffixes=('', '_opp'))
        thisyear_swapped['M/F'] = thisyear_swapped['M/F'].replace({'M': 'F', 'F': 'M'})
        thisyear_swapped['thisyear_count_opp'] = thisyear_swapped['thisyear_count_opp'].fillna(0)
        thisyear = thisyear_swapped
        # display(thisyear)

        df = medians.merge(thisyear, how='left', on=['state', 'name', 'M/F']).rename(columns={'year': 'median_year'})
        # df2[['thisyear_count']] = df2[['thisyear_count']].fillna(0) # might want to shift this to 2 and fill in 2s for missing years? or maybe not

        # this is sort of a rough assumption that if a row didn't exist for this year, not only is the count 0, but so is the diff, shift and pct_change. not always true if the prior year had a count, but often true
        df[['thisyear_count', 'diff', 'shift', 'pct_change', 'accel', 'diff2']] = df[['thisyear_count', 'diff', 'shift', 'pct_change', 'accel', 'diff2']].fillna(0)

        df['after_peak'] = df['after_peak'].fillna(1)
        df['years_since_peak'] = latest_known_year - df['peak_year']

        df['year'] = latest_known_year

        for n in range(1, 4):
            df = df.merge(percentage_of_total_per_year[n], how='left', on=['year', 'state', 'M/F', 'first_letter_'+str(n)])
            df = df.merge(percentage_change_per_year[n], how='left', on=['year', 'state', 'M/F', 'first_letter_'+str(n)])
            df['first_letter_'+str(n)+'_pct'] = df['first_letter_'+str(n)+'_pct'].fillna(0)
            df['first_letter_'+str(n)+'_pct_change'] = df['first_letter_'+str(n)+'_pct_change'].fillna(0)

        # display(df2)
        # display(df2.groupby(['state','name','M/F']).ngroups)

        # change M/F to 0/1 so it works with various models
        # (even HistGradientBoostingRegressor, which accepts categorical values,
        # still needs those values to be numbers not strings)
        df['M/F'] = df['M/F'].map(self.gender_encoding)

        return df
    
    def fit(self, historical_data, first_year_to_predict, years_to_fit=1, weight_decay=0.99):
        # first, we do need to find our y
        historical_data = classify(historical_data)
        
        # first things first, we don't want to know about future data
        historical_data = historical_data[historical_data['year'] < first_year_to_predict]
        # at this point the data we don't want to know should be inaccessible

        # maybe there is a better way to do the following
        # but basically we just want the original columns for X and one new labeled column for y
        X_orig = historical_data[['state', 'year', 'name', 'M/F', 'count']]
        y_orig = historical_data[['state', 'year', 'name', 'M/F', 'has_peaked']]
        y_orig = y_orig.rename(columns={'has_peaked': 'y'})
        y_orig['M/F'] = y_orig['M/F'].map(self.gender_encoding)

        X_all = pd.DataFrame()
        y_all = pd.Series()

        # each year_to_fit is the year that's essentially our y for that loop
        for year_to_fit in range(first_year_to_predict - years_to_fit, first_year_to_predict):

            # now we "know" even less for X
            X = X_orig[X_orig['year'] <= year_to_fit]
            # y = historical_data[historical_data['year'] == year_to_fit]

            X = self.preprocess(X, latest_known_year=year_to_fit)
            # y = y[['state', 'name', 'M/F', 'count']].rename(columns={'count': 'y'})

            # display(X)

            data = X.merge(y_orig, how='left', on=['state', 'year', 'name', 'M/F'])
            data['y'] = data['y'].fillna(1) # if nothing for this name for this year, it has already peaked, so 1

            # assert data['y'].isna().any() == False

            # display(data)

            X = data.drop(columns=['y'])
            y = data['y']
            X['sample_weight'] = weight_decay ** (first_year_to_predict - year_to_fit)

            X_all = pd.concat([X_all, X], ignore_index=True)
            y_all = pd.concat([y_all, y], ignore_index=True)
        
        temp = X_all.copy()
        temp['y'] = y_all
        display(temp)
        display(temp.columns)

        sample_weights = X_all['sample_weight']
        X_all = X_all.drop(columns=['sample_weight'])

        self.pipe.fit(X_all, y_all, **{'histgradientboostingclassifier__sample_weight': sample_weights})
        # this seems like a silly way to pass params to individual steps of the pipeline, but it's true. See: https://stackoverflow.com/questions/36205850/sklearn-pipeline-applying-sample-weights-after-applying-a-polynomial-feature-t

    def predict(self, historical_data):

        # all_known_names = get_all_known_names(historical_data)

        # predictions = []

        # display('historical_data in predict:')
        # display(historical_data)
        # display('years_to_predict:')
        # display(years_to_predict)

        most_recent_year = historical_data['year'].max()

        # predictions = all_known_names.copy()

        # most_recent_year_data = historical_data[historical_data['year'] == most_recent_year]
        # predictions = predictions.merge(most_recent_year_data, how='left', on=['state', 'name', 'M/F'])
        # display(predictions)

        # display('historical_data in predict loop:')
        # display(historical_data)

        df = self.preprocess(historical_data, latest_known_year=most_recent_year)
        # df = self.preprocess(historical_data, years_to_predict[0] - 1)

        df = df[df['year'] == most_recent_year]

        df['y'] = self.pipe.predict(df)

        # df['year'] = year_to_predict
        # display(df)

        # if we want to simply, do the following; 
        # but for now, might be useful to see all data displayed.
        # df = df[['state', 'year', 'name', 'M/F', 'y']]

        predictions = df.copy()

        '''
        assumed_new_year_of_historical_data = df[['state', 'year', 'name', 'M/F', 'y']].rename(columns={'y': 'count'})
        assumed_new_year_of_historical_data['M/F'] = assumed_new_year_of_historical_data['M/F'].map({v: k for k, v in self.gender_encoding.items()})
        historical_data = pd.concat([historical_data, assumed_new_year_of_historical_data], ignore_index=True)
        '''

        # predictions = pd.concat(predictions, ignore_index=True)

        # we have to reverse the mapping to send our predictions
        # (at least the way we currently have it set up)
        predictions['M/F'] = predictions['M/F'].map({v: k for k, v in self.gender_encoding.items()})

        # predictions.loc[predictions['y'] < 4.5, 'y'] = 2

        display(predictions)
        # display(predictions[predictions['y'] < 4.5])

        return predictions

In [18]:
first_year_to_predict = 1993
years_to_predict_in_fitting = 15
years_to_fit_in_fitting = 30
name_usage_cutoff = 100
my_predictor = MyPredictor()
data_to_fit = select_top_names(trainval, first_year_to_predict=first_year_to_predict, cutoff=name_usage_cutoff)
data_to_eval = select_top_names(test, first_year_to_predict=first_year_to_predict, cutoff=name_usage_cutoff)
my_predictor.fit(historical_data=data_to_fit, first_year_to_predict=first_year_to_predict-years_to_predict_in_fitting, years_to_fit=years_to_fit_in_fitting)
evaluate(predictor=my_predictor, data_held_out=data_to_eval, first_year_to_predict=first_year_to_predict, metric='accuracy')

,state,median_year,name,M/F,peak_year,has_peaked,sum,first_letter_1,first_letter_2,first_letter_3,...,years_since_peak,year,first_letter_1_pct,first_letter_1_pct_change,first_letter_2_pct,first_letter_2_pct_change,first_letter_3_pct,first_letter_3_pct_change,sample_weight,y
0,US,1937,Jerimiah,0,1937,1,5,j,je,jer,...,11,1948,0.168296,-0.021434,0.019686,-0.016231,0.012099,-0.057831,0.7397,1.0
1,US,1937,Adonis,0,1946,0,10,a,ad,ado,...,2,1948,0.035509,0.007770,0.000517,0.022720,0.000133,0.257483,0.7397,1.0
2,US,1937,Gage,0,1942,0,10,g,ga,gag,...,6,1948,0.055321,-0.015659,0.018627,-0.010274,0.000000,0.000000,0.7397,1.0
3,US,1937,Nikolas,0,1937,1,6,n,ni,nik,...,11,1948,0.007727,-0.017184,0.001596,-0.014660,0.000000,0.000000,0.7397,1.0
4,US,1937,Claribel,1,1937,1,18,c,cl,cla,...,11,1948,0.094005,0.012204,0.004255,-0.040156,0.004063,-0.036637,0.7397,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112033,US,1977,Destinee,1,1977,1,39,d,de,des,...,0,1977,0.038961,-0.070074,0.012544,-0.057271,0.000947,0.159930,0.9900,0.0
112034,US,1977,Najee,0,1977,1,5,n,na,naj,...,0,1977,0.016569,0.169047,0.008202,0.087693,0.000003,0.000000,0.9900,0.0
112035,US,1977,Itzel,1,1977,1,5,i,it,itz,...,0,1977,0.003076,-0.062262,0.000004,0.000000,0.000004,0.000000,0.9900,0.0
112036,US,1977,Breeanna,1,1977,1,12,b,br,bre,...,0,1977,0.028791,-0.005786,0.014976,0.020864,0.002782,-0.106124,0.9900,0.0


Index(['state', 'median_year', 'name', 'M/F', 'peak_year', 'has_peaked', 'sum',
       'first_letter_1', 'first_letter_2', 'first_letter_3', 'median_age',
       'thisyear_count', 'diff', 'shift', 'pct_change', 'accel', 'diff2',
       'after_peak', 'thisyear_count_opp', 'years_since_peak', 'year',
       'first_letter_1_pct', 'first_letter_1_pct_change', 'first_letter_2_pct',
       'first_letter_2_pct_change', 'first_letter_3_pct',
       'first_letter_3_pct_change', 'sample_weight', 'y'],
      dtype='object')

,state,name,M/F,median_year,peak_year,has_peaked,sum,first_letter_1,first_letter_2,first_letter_3,...,thisyear_count_opp,years_since_peak,year,first_letter_1_pct,first_letter_1_pct_change,first_letter_2_pct,first_letter_2_pct_change,first_letter_3_pct,first_letter_3_pct_change,y
0,US,Shelva,F,1940,1937,1,1028,s,sh,she,...,NaN,55,1992,0.092766,-0.007713,0.021895,0.022871,0.008309,-0.057600,1.0
1,US,Shelba,F,1941,1937,1,1964,s,sh,she,...,NaN,55,1992,0.092766,-0.007713,0.021895,0.022871,0.008309,-0.057600,1.0
2,US,Melvyn,M,1942,1938,1,3013,m,me,mel,...,0.0,54,1992,0.083813,-0.049057,0.000596,-0.049408,0.000450,-0.053206,1.0
3,US,Dick,M,1944,1937,1,14058,d,di,dic,...,0.0,55,1992,0.091003,0.064203,0.004170,0.360184,0.000006,0.595606,1.0
4,US,Myrtle,F,1944,1937,1,15448,m,my,myr,...,0.0,55,1992,0.096376,0.013237,0.000490,-0.043273,0.000403,-0.026740,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4421,US,Jaylen,M,1992,1992,1,109,j,ja,jay,...,0.0,0,1992,0.179265,-0.043854,0.049668,0.003470,0.001045,0.015905,0.0
4422,US,Payton,F,1992,1992,1,560,p,pa,pay,...,262.0,0,1992,0.011322,0.055696,0.007973,0.061277,0.000185,5.955524,0.0
4423,US,Connor,F,1992,1992,1,236,c,co,con,...,4607.0,0,1992,0.090971,0.013571,0.012252,-0.024929,0.000624,0.003406,0.0
4424,US,Bronte,F,1992,1992,1,185,b,br,bro,...,0.0,0,1992,0.055148,-0.014464,0.044932,-0.012426,0.004079,0.110663,0.0


Score: 0.8436511522819702


,state,name,M/F,year_true,count,peak_year_true,has_peaked_true,y_true,median_year,peak_year_pred,...,thisyear_count_opp,years_since_peak,year_pred,first_letter_1_pct,first_letter_1_pct_change,first_letter_2_pct,first_letter_2_pct_change,first_letter_3_pct,first_letter_3_pct_change,y_pred
0,US,Aaron,F,1992.0,91.0,1980.0,1.0,1.0,1980,1980,...,14509.0,12,1992,0.143005,-0.033572,0.000055,-0.087438,0.000055,-0.087438,0.0
1,US,Aaron,M,1992.0,14509.0,1989.0,1.0,1.0,1981,1989,...,91.0,3,1992,0.091399,0.036444,0.007758,0.034494,0.007758,0.034494,1.0
2,US,Abbey,F,1992.0,431.0,1999.0,0.0,0.0,1985,1990,...,0.0,2,1992,0.143005,-0.033572,0.003535,0.025855,0.001072,-0.046038,1.0
3,US,Abbie,F,1992.0,260.0,2003.0,0.0,0.0,1979,1990,...,0.0,2,1992,0.143005,-0.033572,0.003535,0.025855,0.001072,-0.046038,0.0
4,US,Abby,F,1992.0,1081.0,2003.0,0.0,0.0,1983,1983,...,0.0,9,1992,0.143005,-0.033572,0.003535,0.025855,0.001072,-0.046038,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4421,US,Zelma,F,1992.0,8.0,1938.0,1.0,1.0,1948,1938,...,0.0,54,1992,0.000862,0.337240,0.000050,0.035345,0.000019,-0.039121,1.0
4422,US,Zena,F,1992.0,52.0,1964.0,1.0,1.0,1965,1964,...,0.0,28,1992,0.000862,0.337240,0.000050,0.035345,0.000031,0.085496,1.0
4423,US,Zina,F,1992.0,21.0,1964.0,1.0,1.0,1964,1964,...,0.0,28,1992,0.000862,0.337240,0.000013,-0.259300,0.000013,-0.259300,1.0
4424,US,Zoe,F,1992.0,982.0,2012.0,0.0,0.0,1981,1992,...,0.0,0,1992,0.000862,0.337240,0.000682,0.409261,0.000682,0.409261,0.0
